In [1]:
import numpy as np, cPickle, sys, re, pandas as pd
from collections import defaultdict

## Formats

- `mr.p`

```
[ revs, W, W2, word_idx_map, vocab ]
```

- `revs`: 
- `W`: word2vec vectors
- `W2`: random vectors
- `word_idx_map`: word index map
- `vocab`: vocabularies